In [26]:
from __future__ import print_function
%matplotlib inline
import fluiddyn as fld

# Tutorial: running a simulation (user perspective)

In this tutorial, I'm going to show how to run a simple simulation with a solver already written that solves the 2 dimensional Navier-Stokes equations. I'm also going to present some useful concepts and objects used in the simulation part of FluidDyn.

## Importing a solver

In [27]:
from fluidsim.solvers.ns2d import solver
solver.__file__

'/home/pierre/Dev/fluidsim/fluidsim/solvers/ns2d/solver.py'

There is also another quite convenient way to import a solver from a string, here "NS2D":

In [28]:
solver = fld.simul.import_module_solver_from_key('NS2D')
solver.__file__

'/home/pierre/Dev/fluidsim/fluidsim/solvers/ns2d/solver.py'

## Create an instance of the class Parameters

The next step is to create an object `params` from the information contained in the module `solver`:

In [29]:
params = solver.Simul.create_default_params()

In [30]:
[attr for attr in dir(params) if not attr.startswith(('_', 'set_', 'xml_'))]

['FORCING',
 'NEW_DIR_RESULTS',
 'ONLY_COARSE_OPER',
 'beta',
 'forcing',
 'init_fields',
 'nu_2',
 'nu_4',
 'nu_8',
 'nu_m4',
 'oper',
 'output',
 'short_name_type_run',
 'time_stepping']

and some useful functions (whose names all start with 'set_' or 'xml_'): 

In [31]:
[attr for attr in dir(params) if attr.startswith(('set_', 'xml_'))]

['set_as_child',
 'set_attrib',
 'set_attribs',
 'set_child',
 'xml_attrib',
 'xml_print',
 'xml_produce_text',
 'xml_save',
 'xml_tag',
 'xml_tag_children',
 'xml_to_hdf5']

In [32]:
print(type(params.nu_2))
print(type(params.output))

<type 'float'>
<class 'fluidsim.base.params.Parameters'>


In [33]:
[attr for attr in dir(params.output) if not attr.startswith(('_', 'set_', 'xml_'))]

['HAS_TO_SAVE',
 'ONLINE_PLOT_OK',
 'increments',
 'period_show_plot',
 'periods_plot',
 'periods_print',
 'periods_save',
 'phys_fields',
 'spatial_means',
 'spect_energy_budg',
 'spectra']

Therefore, the object `params` contains a tree of parameters. This tree can be represented as xml code:

In [34]:
print(params)


<params nu_2="0.0" ONLY_COARSE_OPER="False" short_name_type_run=""
        FORCING="False" NEW_DIR_RESULTS="True" nu_m4="0.0" nu_8="0.0" nu_4="0.0"
        beta="0.0">
  <oper ny="48" nx="48" type_fft="FFTWCY" coef_dealiasing="0.6666666666666666"
        TRANSPOSED_OK="True" Lx="8" Ly="8"/>  

  <init_fields type_flow_init="NOISE" path_file="" max_velo_noise="1.0"
               lambda_noise="1.0"/>  

  <time_stepping type_time_scheme="RK4" it_end="10" USE_CFL="True" deltat0="0.2"
                 USE_T_END="True" t_end="10.0"/>  

  <output ONLINE_PLOT_OK="True" period_show_plot="1" HAS_TO_SAVE="True">
    <periods_save increments="0" phys_fields="0" spectra="0" spatial_means="0"
                  spect_energy_budg="0"/>  

    <periods_print print_stdout="1.0"/>  

    <periods_plot phys_fields="0"/>  

    <increments HAS_TO_PLOT_SAVED="False"/>  

    <phys_fields field_to_plot="rot"/>  

    <spectra HAS_TO_PLOT_SAVED="False"/>  

    <spatial_means HAS_TO_PLOT_SAVED="False"/>  

## Set the parameters for your simulation

The user can change any parameters

In [35]:
params.nu_2 = 1.
params.FORCING = False
params.init_fields.type_flow_init

'NOISE'

In [36]:
try:
    params.this_param_does_not_exit = 10
except KeyError as e:
    print('KeyError:', e)

KeyError: 'this_param_does_not_exit is not a proper key.'


This behaviour is much safer than using a text file or a python file for the parameters.   

## Instantiate a simulation object

The next step is to create a simulation object (instance of the class solver.Simul) with the parameters in `params`:

In [37]:
sim = solver.Simul(params)

Init. operator
*************************************
Program FluidDyn

solver NS2D, RK4 and sequenciel,
nx =     48 ; ny =     48
Lx = 8. ; Ly = 8.
path_run =
/home/pierre/Sim_data/NS2D_L=8.x8._48x48_2015-04-03_17-56-07
type_flow_init = NOISE
Initialization outputs:
<class 'fluidsim.base.output.increments.Increments'> increments
<class 'fluidsim.base.output.phys_fields.PhysFieldsBase'> phys_fields
<class 'fluidsim.solvers.ns2d.output.spectra.SpectraNS2D'> spectra
<class 'fluidsim.solvers.ns2d.output.spatial_means.SpatialMeansNS2D'> spatial_means
<class 'fluidsim.solvers.ns2d.output.spect_energy_budget.SpectralEnergyBudgetNS2D'> spect_energy_budg

Memory usage at the end of init. (equiv. seq.): 82.69921875 Mo
Size of state_fft (equiv. seq.): 0.0192 Mo


which initializes everything needed to run the simulation. The object `sim` has few attributes:

In [38]:
[attr for attr in dir(sim) if not attr.startswith('_')]

['InfoSolver',
 'compute_freq_diss',
 'create_default_params',
 'info',
 'info_solver',
 'init_fields',
 'name_run',
 'oper',
 'output',
 'params',
 'state',
 'tendencies_nonlin',
 'time_stepping']

In [39]:
print(sim.info.__class__)
print([attr for attr in dir(sim.info) if not attr.startswith(('_', 'set_', 'xml_'))])

<class 'fluiddyn.util.containerxml.ContainerXML'>
['params', 'solver']


In [40]:
print(sim.info.solver)


<solver class_name="Simul" module_name="fluidsim.solvers.ns2d.solver"
        short_name="NS2D">
  <classes>
    <InitFields class_name="InitFieldsNS2D"
                module_name="fluidsim.solvers.ns2d.init_fields"/>  

    <Forcing class_name="ForcingNS2D"
             module_name="fluidsim.solvers.ns2d.forcing">
      <classes>
        <Random class_name="Random"
                module_name="fluidsim.solvers.ns2d.forcing"/>  

        <Proportional class_name="Proportional"
                      module_name="fluidsim.solvers.ns2d.forcing"/>  

      </classes>

    </Forcing>

    <Output class_name="Output" module_name="fluidsim.solvers.ns2d.output">
      <classes>
        <PrintStdOut class_name="PrintStdOutNS2D"
                     module_name="fluidsim.solvers.ns2d.output.print_stdout"/>  

        <PhysFields class_name="PhysFieldsBase"
                    module_name="fluidsim.base.output.phys_fields"/>  

        <Spectra class_name="SpectraNS2D"
                 module_n

## Run the simulation

We are ready to launch the simulation!

In [41]:
sim.time_stepping.start()

*************************************
Beginning of the computation
save state_phys in file state_phys_t=000.000.hd5
    compute until t =         10
it =      1 ; t      =     0.097 ; deltat       =     0.1499
              energy = 4.033e-02 ; Delta energy = -5.126e-02
              estimated remaining duration =      0 s
it =      7 ; t      =     1.247 ; deltat       =        0.2
              energy = 1.938e-03 ; Delta energy = -3.840e-02
              estimated remaining duration =      0 s
it =     13 ; t      =     2.447 ; deltat       =        0.2
              energy = 3.661e-04 ; Delta energy = -1.572e-03
              estimated remaining duration =      0 s
it =     18 ; t      =     3.447 ; deltat       =        0.2
              energy = 1.024e-04 ; Delta energy = -2.637e-04
              estimated remaining duration =      0 s
it =     23 ; t      =     4.447 ; deltat       =        0.2
              energy = 2.945e-05 ; Delta energy = -7.291e-05
              estimated r